In [1]:
import pandas as pd
import numpy as np
from tsfresh.feature_extraction import extract_features, EfficientFCParameters
from tsfresh.utilities.dataframe_functions import impute
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import import_ipynb

from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report, balanced_accuracy_score

In [2]:
raw_data_prep_dir = '../raw_data/preprocessed/'

In [3]:
file_list = pd.read_csv(raw_data_prep_dir  + "file_list.csv")
file_list

,resource_type,id,study_id,condition,disease_comment,age_at_diagnosis,age,height,weight,gender,handedness,appearance_in_kinship,appearance_in_first_grade_kinship,effect_of_alcohol_on_tremor,label
0,patient,1,PADS,Healthy,-,56,56,173,78,male,right,True,True,Unknown,0
1,patient,2,PADS,Other Movement Disorders,Left-Sided resting tremor and hypokinesia with...,69,81,193,104,male,right,False,NaN,No effect,2
2,patient,3,PADS,Healthy,-,45,45,170,78,female,right,False,NaN,Unknown,0
3,patient,4,PADS,Parkinson's,IPS akinetic-rigid type,63,67,161,90,female,right,False,NaN,No effect,1
4,patient,5,PADS,Parkinson's,IPS tremordominant type,65,75,172,86,male,left,False,NaN,Unknown,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
464,patient,465,PADS,Parkinson's,IPS mixed type,62,65,175,80,male,right,True,False,No effect,1
465,patient,466,PADS,Healthy,-,84,84,172,74,female,right,True,True,No effect,0
466,patient,467,PADS,Parkinson's,"Essential Tremor, starting IPS tremordominant ...",55,57,190,100,male,right,False,NaN,Improvement,1
467,patient,468,PADS,Parkinson's,IPS mixed type,73,76,198,118,male,right,False,NaN,No effect,1


In [4]:
n_id = len(file_list['id'])
n_id

469

In [5]:
file_list[file_list['id']==5]['label']

4    1
Name: label, dtype: int64

In [6]:
time_data = []
y = []
X_to_split = []
for idx, subject_idx in enumerate(file_list['id']):
    file_name = raw_data_prep_dir + f'movement/{subject_idx:03d}_ml.bin'
    time_idx_data = np.fromfile(file_name, dtype=np.float32).reshape((-1, 976))
    time_data.append(time_idx_data)
    y_idx = file_list[file_list['id']==subject_idx].label.values
    X_id = idx
    y.append(y_idx)
    X_to_split.append(X_id)

X_to_split = np.array(X_to_split)
y = np.array(y).flatten()
print(X_to_split.shape, y.shape)


(469,) (469,)


value_cols = ['x','y','z','speed','acc','ang_vel']
fc = EfficientFCParameters()  # compact, fast set
X = extract_features(df[['traj_id','t'] + value_cols],
                     column_id='traj_id', column_sort='t',
                     default_fc_parameters=fc, disable_progressbar=True)

In [7]:
channels = []
for task in ["Relaxed1", "Relaxed2", "RelaxedTask1", "RelaxedTask2", "StretchHold", "HoldWeight",
                "DrinkGlas", "CrossArms", "TouchNose", "Entrainment1", "Entrainment2"]:
    for device_location in ["LeftWrist", "RightWrist"]:
        for sensor in ["Acceleration", "Rotation"]:
            for axis in ["X", "Y", "Z"]:
                channel = f"{task}_{sensor}_{device_location}_{axis}"
                channels.append(channel)
len(channels)

132

In [8]:
time_data_array = np.array(time_data)
print(time_data_array.shape)

(469, 132, 976)


In [9]:
X_train_idx, X_test_idx, y_train, y_test = train_test_split(X_to_split, y, test_size=0.20, random_state=42, stratify = y)

In [10]:
X_train = time_data_array[X_train_idx]
X_test = time_data_array[X_test_idx]
print(len(X_train), len(X_test))
print(len(y_train), len(y_test))
X = time_data_array

375 94
375 94


In [11]:
X_train.shape

(375, 132, 976)

In [ ]:
import numpy as np
import pandas as pd
from tsfresh.feature_extraction import extract_features, MinimalFCParameters

# shape (N, F, T)
N, F, T = X_test.shape
feature_names = [f"feat_{i}" for i in range(F)]

def make_long_df(batch_data: np.ndarray, id_offset: int = 0) -> pd.DataFrame:

    n_batch = batch_data.shape[0]


    x = np.transpose(batch_data, (0, 2, 1))
    values = x.reshape(-1)

    ids = np.repeat(np.arange(id_offset, id_offset + n_batch), T * F)


    times = np.tile(np.repeat(np.arange(T), F), n_batch)


    kinds_idx = np.tile(np.arange(F), n_batch * T)
    kinds = np.array(feature_names, dtype=object)[kinds_idx]

    return pd.DataFrame({"id": ids, "time": times, "kind": kinds, "value": values})

df_long = make_long_df(X, id_offset=0)


X_features = extract_features(
    df_long,
    column_id="id",
    column_sort="time",
    column_kind="kind",
    column_value="value",
    default_fc_parameters=MinimalFCParameters(),
    n_jobs=0
)

print(X_features.shape)  # (375, num_features)

Feature Extraction: 100%|██████████| 61908/61908 [00:11<00:00, 5417.31it/s]


(469, 1320)


In [13]:
X_train, X_test, y_train, y_test = train_test_split(X_features, y, test_size=0.20, random_state=42, stratify = y)


In [14]:
base_model_xgb = XGBClassifier()
base_model_xgb.fit(X_train,y_train)

y_pred = base_model_xgb.predict(X_test)


# Metrics
print("Balanced Accuracy:", balanced_accuracy_score(y_test, y_pred))

print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Balanced Accuracy: 0.4551383399209486

Confusion Matrix:
 [[ 4 12  0]
 [ 1 47  7]
 [ 0 17  6]]

Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.25      0.38        16
           1       0.62      0.85      0.72        55
           2       0.46      0.26      0.33        23

    accuracy                           0.61        94
   macro avg       0.63      0.46      0.48        94
weighted avg       0.61      0.61      0.57        94

